In [1]:
!pip install --extra-index-url=https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/14253903 azureml-defaults==0.1.0.14253903 azureml-pipeline-wrapper==0.1.0.14253903 azureml-designer-core --user --upgrade

/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl (82kB)
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 768kB 65.3MB/s 
     |████████████████████████████████| 153kB 60.7MB/s 
     |████████████████████████████████| 2.7MB 70.7MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 727kB 54.9MB/s 
     |████████████████████████████████| 92kB 7.5MB/s 
     |████████████████████████████████| 143kB 63.3MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 133kB 60.4MB/s 
     |████████████████████████████████| 40kB 3.7MB/s 
     |████████████████████████████████| 542kB 73.5MB/s 
     |████████████████████████████████| 8.6MB 806kB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
    

In [3]:
!python --version

Python 3.6.10 :: Anaconda, Inc.


In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

#train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='MPI Train')
train_module_func = Module.load_from_yaml(ws, './train-score-eval/mpi_train.yaml')
score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')
compare_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Compare 2 Models')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [2]:
# define a sub pipeline
@dsl.pipeline(name = 'Train-Score-Eval', 
              description = 'train model and evaluate model perf')
def training_pipeline(input_data, learning_rate):
    train = train_module_func(
        training_data=input_data, 
        max_epochs=5, 
        learning_rate=learning_rate)
   
    train.runsettings.configure(process_count_per_node = 2, node_count = 2)

    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)
    
    return {'eval_output': eval.outputs.eval_output, 'model_output': train.outputs.model_output}

In [3]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'Dummy automl pipeline', 
              description = 'select best model trained with different learning rate',
              default_compute_target = 'cpu-cluster')
def dummy_automl_pipeline():
    train_and_evalute_model1 = training_pipeline(train_data, 0.01)
    train_and_evalute_model2 = training_pipeline(train_data, 0.02)
    
    compare = compare_module_func(
        model1=train_and_evalute_model1.outputs.model_output, 
        eval_result1=train_and_evalute_model1.outputs.eval_output,
        model2=train_and_evalute_model2.outputs.model_output,
        eval_result2=train_and_evalute_model2.outputs.eval_output
    )

    return {**compare.outputs}

# create a pipeline
pipeline = dummy_automl_pipeline()

In [4]:
pipeline.validate()

{'result': 'validation passed', 'errors': []}

In [11]:
pipeline.submit(experiment_name='samples')

Submitted PipelineRun 595848f9-6e47-47cf-9bc9-0975e0a73fd4
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/595848f9-6e47-47cf-9bc9-0975e0a73fd4?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1


Experiment,Id,Type,Status,Details Page,Docs Page
samples,595848f9-6e47-47cf-9bc9-0975e0a73fd4,azureml.PipelineRun,Running,Link to Azure Machine Learning studio,Link to Documentation
